In [ ]:
import tensorflow as tf

import scipy

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 구글 드라이브 셋팅 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -rf "sample_data/" "kfood50_imageSet/" # 파일 삭제하는 커맨드이므로 주의할 것!

In [ ]:
!unzip "/content/drive/MyDrive/AI_images/kfood50_imageSet.zip" # 이미지셋 다운로드

In [ ]:
## 이미지셋 로드시 필요한 변수 지정
imgSet_path = 'kfood50_imageSet/'
train_path = imgSet_path + 'train/'
valid_path = imgSet_path + 'validation/'
test_path = imgSet_path + 'test/'

img_height = 224
img_width = 224
batch_size = 32;

In [ ]:
datagen = ImageDataGenerator(rescale=1./ 255)
train_datagen = ImageDataGenerator(
                    rescale=1./ 255,
                    rotation_range=10,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                )

# 학습 데이터셋 - 이미지 증강 적용
train_generator = train_datagen.flow_from_directory(
    train_path,
    shuffle=True,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# 검증 데이터셋 - 학습시 loss, accuracy 확인 
valid_generator = datagen.flow_from_directory(
    valid_path,
    shuffle=True,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# 테스트 데이터셋 - 학습된 모델 정확도 확인
test_generator = datagen.flow_from_directory(
    test_path,
    shuffle=False,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 37500 images belonging to 50 classes.
Found 7500 images belonging to 50 classes.
Found 5000 images belonging to 50 classes.


In [ ]:
##### 음식 레이블 json 형식으로 저장
import json

# 음식 레이블 저장
class_dict = test_generator.class_indices # dict 형태 : {음식 종류 : 종류 index}
print('음식 종류 :' ,list(class_dict.keys())) # 음식 이름 확인
print('클래스 개수 :', len(class_dict.keys()))

# JSON 형식으로 저장 - {index : 음식종류} 형태
jsonfile_name = "food_classes.json"
rev_class_dict = {v : k for k, v in class_dict.items()}
with open(jsonfile_name, "w", encoding="utf-8") as json_file:
    json.dump(rev_class_dict, json_file, ensure_ascii=False)

<class 'dict'>
dict_keys(['고사리나물무침', '달걀국', '닭개장', '삼겹살구이', '소고기무국', '숙주나물무침', '시금치나물무침', '시래기된장국', '훈제오리', '갈비탕', '갈치구이', '감자조림', '감자탕', '건새우볶음', '고등어구이', '김치볶음밥', '김치찌개', '누룽지', '동태찌개', '된장찌개', '두부김치', '두부조림', '떡갈비', '떡볶이', '멸치볶음', '물냉면', '미역국', '북엇국', '비빔냉면', '비빔밥', '삼계탕', '새우볶음밥', '새우튀김', '소세지볶음', '순두부찌개', '알밥', '애호박볶음', '어묵볶음', '오징어튀김', '유부초밥', '육개장', '잡곡밥', '제육볶음', '주꾸미볶음', '짜장면', '짬뽕', '쫄면', '칼국수', '콩나물국', '황태구이'])
클래스 개수 :  50


In [ ]:
!cp $jsonfile_name /content/drive/MyDrive/importFiles/$jsonfile_name # 드라이브에 저장

In [ ]:
# json 파일 정상 오픈 확인 -> 서버에서 import할 모듈 작성 시 참고
import json

jsonfile_path = '/content/drive/MyDrive/importFiles/'
jsonfile_name = "food_classes.json"
with open(jsonfile_path+jsonfile_name, "r", encoding="utf-8") as json_file:
    jsonDict = json.load(json_file)

# type 및 read 확인
print(type(list(jsonDict.keys())[0]), type(list(jsonDict.values())[0]))
for k, v in jsonDict.items():
    print (int(k), v)